In [2]:
import pandas as pd
import dill
import plotly.express as px
pd.options.plotting.backend = "plotly"

diffs = pd.read_pickle('diff_from_schedule.pkl')

In [ ]:
# what part of delay is explained by overcrowding vs waiting on the train in front of you?
# how to tell if train is waiting on train at next station?
# what is the buffer baked into the schedule? Can we lower it safely?
# markov model for joint distribution (I think the problem has a natural markov structure since the last
# delay will include the only part of the total delay we care about)
# what is the operating advice we can give MTA?
# can we compare performance to post covid?
# can we estimate arrival time of next train?
# P(Arrival = T | station, last train arrival). Also should be markov?
# what is difference between markov and renewal process?
# renewal process have to have IID dist inter event intervals?


In [224]:
afternoon = pd.read_pickle('afternoon.pkl').drop(0, axis=1)
px.line(afternoon.loc['2020-01-16':'2020-01-16 15:00:00', :].transpose())

In [45]:
diffs.columns = diffs.columns.astype('int32')
diffs.to_pickle('diff_from_schedule.pkl')

In [47]:
second_diffs = diffs.apply(lambda c: c.dt.total_seconds(), axis=0).drop(0, axis=1)
second_diffs.to_pickle('second_diffs.pkl')

In [62]:
ontime = pd.read_pickle('trip.pkl')+ pd.to_datetime('2020-08-04')
ontime['stop_time']

0    2020-08-04 11:51:30
1    2020-08-04 11:54:00
2    2020-08-04 11:55:00
3    2020-08-04 11:56:00
4    2020-08-04 11:57:00
5    2020-08-04 11:58:00
6    2020-08-04 12:00:00
7    2020-08-04 12:01:30
8    2020-08-04 12:03:00
9    2020-08-04 12:04:00
10   2020-08-04 12:05:00
11   2020-08-04 12:06:00
12   2020-08-04 12:07:00
13   2020-08-04 12:08:30
14   2020-08-04 12:10:30
15   2020-08-04 12:12:30
16   2020-08-04 12:14:00
17   2020-08-04 12:15:00
18   2020-08-04 12:17:00
19   2020-08-04 12:18:00
20   2020-08-04 12:20:00
21   2020-08-04 12:21:30
22   2020-08-04 12:23:00
23   2020-08-04 12:24:00
24   2020-08-04 12:26:30
25   2020-08-04 12:28:30
26   2020-08-04 12:30:00
27   2020-08-04 12:32:00
28   2020-08-04 12:33:30
29   2020-08-04 12:35:30
30   2020-08-04 12:37:00
31   2020-08-04 12:38:00
32   2020-08-04 12:39:30
33   2020-08-04 12:41:00
34   2020-08-04 12:42:30
35   2020-08-04 12:44:00
36   2020-08-04 12:45:30
37   2020-08-04 12:49:30
Name: stop_time, dtype: datetime64[ns]

In [104]:
stops = pd.read_csv('stop_names.csv', index_col='stop_index')

1                        Rector St
2                    WTC Cortlandt
3                      Chambers St
4                      Franklin St
5                         Canal St
6                       Houston St
7     Christopher St - Sheridan Sq
8                            14 St
9                            18 St
10                           23 St
11                           28 St
12            34 St - Penn Station
13                Times Sq - 42 St
14                           50 St
15         59 St - Columbus Circle
16          66 St - Lincoln Center
17                           72 St
18                           79 St
19                           86 St
20                           96 St
21                          103 St
22                  Cathedral Pkwy
23    116 St - Columbia University
24                          125 St
25           137 St - City College
26                          145 St
27                          157 St
28         168 St - Washington Hts
29                  

In [126]:
with_counts = second_diffs.copy()
# add these for sorting
with_counts['total_delay'] = second_diffs[(second_diffs > 0)].sum(axis=1)/60
with_counts['num_delays'] = (second_diffs > 0).sum(axis=1)

Index([            1,             2,             3,             4,
                   5,             6,             7,             8,
                   9,            10,            11,            12,
                  13,            14,            15,            16,
                  17,            18,            19,            20,
                  21,            22,            23,            24,
                  25,            26,            27,            28,
                  29,            30,            31,            32,
                  33,            34,            35,            36,
                  37, 'total_delay',  'num_delays'],
      dtype='object')

In [131]:
sorted_diffs = with_counts.sort_values('num_delays').drop(['num_delays', 'total_delay'], axis=1)

In [180]:
px.imshow(sorted_diffs[(sorted_diffs.isna().sum(axis=1) < 10)].iloc[200:300, :] / 60).write_html('heatmap_delays.html', auto_open=True)

In [204]:
median_trip = pd.to_timedelta(sorted_diffs.median(axis=0), unit='s')
mean_trip = pd.to_timedelta(sorted_diffs.mean(axis=0), unit='s')

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
median,NaT,2020-08-04 11:52:31,2020-08-04 11:53:44,2020-08-04 11:54:59,2020-08-04 11:56:18,2020-08-04 11:57:26,2020-08-04 11:58:46,2020-08-04 12:00:08,2020-08-04 12:01:26,2020-08-04 12:02:43,...,2020-08-04 12:32:45,2020-08-04 12:34:37,2020-08-04 12:36:09.500,2020-08-04 12:37:34,2020-08-04 12:39:18,2020-08-04 12:40:47,2020-08-04 12:42:22.500,2020-08-04 12:43:46,2020-08-04 12:45:18,2020-08-04 12:47:31.500
ontime,2020-08-04 11:51:30,2020-08-04 11:54:00,2020-08-04 11:55:00,2020-08-04 11:56:00,2020-08-04 11:57:00,2020-08-04 11:58:00,2020-08-04 12:00:00,2020-08-04 12:01:30,2020-08-04 12:03:00,2020-08-04 12:04:00,...,2020-08-04 12:33:30,2020-08-04 12:35:30,2020-08-04 12:37:00.000,2020-08-04 12:38:00,2020-08-04 12:39:30,2020-08-04 12:41:00,2020-08-04 12:42:30.000,2020-08-04 12:44:00,2020-08-04 12:45:30,2020-08-04 12:49:30.000


In [206]:
px.line(pd.DataFrame([mean_trip + ontime['stop_time'], median_trip + ontime['stop_time'], ontime['stop_time']], index=['mean', 'median', 'ontime']).transpose()).write_html('median_path.html', auto_open=True)

In [207]:
px.line(pd.DataFrame([sorted_diffs.mean(axis=0), sorted_diffs.median(axis=0)], index=['mean', 'median']).transpose()).write_html('median_delays.html', auto_open=True)

In [173]:
one_sample = sorted_diffs[(sorted_diffs.isna().sum(axis=1) < 2)].iloc[-60:, :].sample(1).transpose()
one_sample

,2020-02-21 11:51:00
1,-84.0
2,-72.0
3,-52.0
4,-14.0
5,-7.0
6,-46.0
7,-51.0
8,-63.0
9,-46.0
10,-34.0


In [169]:
one_sample.cumsum()

,2020-01-23 12:15:00
1,-93.0
2,-173.0
3,-237.0
4,-286.0
5,-328.0
6,-407.0
7,-493.0
8,-592.0
9,-653.0
10,-703.0


In [175]:
px.line(one_sample).write_html('one_trip.html', auto_open=True)

In [107]:
# have more than 20 stops delayed by more than a minute
trips = afternoon[((second_diffs > 600).sum(axis=1) == 2)].copy()
# put in the reference 'on time' trip
trips.loc['2020-08-04 11:51:00'] = ontime['stop_time'][1:]
trips.columns = stops['stop_name'][trips.columns]
trips.shape

(2, 37)

In [108]:
px.line(trips.apply(lambda r: r + (trips.index[0] - r.index),axis=0).transpose())